# Practice overturning calculation

`cd project/seniorthesis/notebooks`

`conda list PyCO2SYS`

`pip install PyCO2SYS`

`pip install git+https://github.com/jbusecke/cmip6_preprocessing.git`

`mv *.png ../figures`

In [18]:
import xarray as xr
import xesmf as xe
from cmip6_preprocessing.preprocessing import combined_preprocessing
import PyCO2SYS as pyco2 # i have to install this package through terminal for it to work
import intake
import matplotlib.pyplot as plt
import numpy as np
import warnings
import gsw
from cmip6_preprocessing.postprocessing import combine_datasets
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import matplotlib.path as mpath
from cmip6_preprocessing.postprocessing import combine_datasets
from cmip6_preprocessing.postprocessing import merge_variables
from dask.diagnostics import ProgressBar
from xgcm import Grid
from xhistogram.xarray import histogram
import pandas as pd
import scipy

from xgcm.autogenerate import generate_grid_ds

from cmip6_preprocessing.postprocessing import match_metrics


In [2]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

def pick_first_member(ds_list, **kwargs):
    return ds_list[0]

## Just going to get the regular dataset for CESM2-FV2

In [3]:
dd_CESM2FV2 = {}
dd_CESM2FV2['CESM2-FV2.gr.historical.Omon']= xr.open_mfdataset('CESM2-FV2.nc')
for name, ds in dd_CESM2FV2.items():
    # da = dd_x[name]
    ds['p'] = gsw.p_from_z(-1*ds['lev'], ds['y'], geo_strf_dyn_height=0, sea_surface_geopotential=0)*xr.ones_like(ds['talk']) # expand along time dimension
    ds['insitutemp'] = gsw.t_from_CT(ds['so'], ds['thetao'], ds['p'])
    # ds['sigma2'] = gsw.sigma2(da['so'],da['thetao']).isel(x=slice(180,200)).mean('x', keep_attrs=True)
    ds['sigma0'] = gsw.sigma0(ds['so'],ds['thetao'])
    # ds['sigma1'] = gsw.sigma1(da['so'],da['thetao']).isel(x=slice(180,200)).mean('x', keep_attrs=True)

# Functions from Graeme

In [16]:


# Get some more grid information
# (and correct the lev variable
# so that it is actually between
# lev_bounds)

# Separately calculate dx and dy
def _degrees_to_meters(dlon, dlat, lon, lat):
        """Converts lat/lon differentials into distances in meters
        PARAMETERS
        ----------
        dlon : xarray.DataArray longitude differentials
        dlat : xarray.DataArray latitude differentials
        lon  : xarray.DataArray longitude values
        lat  : xarray.DataArray latitude values
        RETURNS
        -------
        dx  : xarray.DataArray distance inferred from dlon
        dy  : xarray.DataArray distance inferred from dlat
        """

        distance_1deg_equator = 111000.0
        dx = dlon * np.cos(np.deg2rad(lat)) * distance_1deg_equator
        dy = ((lon * 0) + 1) * dlat * distance_1deg_equator
        return dx, dy

def get_xgcm(ds,gridlon='x',gridlat='y',gridlev='lev',periodic=None):
    ''' Generate metrics and grid locations'''

    ds = generate_grid_ds(ds, {'X':gridlon,'Y':gridlat})
    xgrid = Grid(ds, periodic=periodic)

    # Get horizontal distances
    dlonG = xgrid.diff(ds[gridlon], 'X', boundary_discontinuity=360)
    dlonC = xgrid.diff(ds[gridlon+'_left'], 'X', boundary_discontinuity=360)

    dlatG = xgrid.diff(ds[gridlat], 'Y', boundary='fill', fill_value=np.nan)
    dlatC = -xgrid.diff(ds[gridlat+'_left'], 'Y', boundary='fill', fill_value=np.nan)

    ds['dxG'], ds['dyG'] = _degrees_to_meters(dlonG, dlatG, ds[gridlon], ds[gridlat])
    ds['dxC'], ds['dyC'] = _degrees_to_meters(dlonC, dlatC, ds[gridlon], ds[gridlat])
    
    # Get vertical distances
    ds['dz'] = ds['lev_bounds'].diff('bnds').squeeze()
    
    # Get area
    ds['areacello'] = ds['dxC']*ds['dyC']
    
    # Regenerate grid
    coords = {
        'X':{'center':gridlon,'left':gridlon+'_left'},
        'Y':{'center':gridlat,'left':gridlat+'_left'},
        'Z':{'center':gridlev,'outer':gridlev+'_outer'}
    }
    metrics = {
        'X':['dxC','dxG'],
        'Y':['dyC','dyG'],
        'Z':['dz'],
        ('X','Y'):['areacello']
    }
    
    xgrid = Grid(ds,coords=coords,metrics=metrics,periodic=periodic)
    
    return ds,xgrid

def get_xgcm_special(ds,gridlon='x',gridlat='y',gridlev='lev',periodic=None):
    ''' Generate metrics and grid locations
        This is a hacked version to specify "outer" coordinates for y,
        for use with transform method.'''

    ds = generate_grid_ds(ds, {'X':gridlon,'Y':gridlat})
    lat_left = np.append(ds[gridlat+'_left'],ds[gridlat+'_left'].isel(y_left=-1)+1)
    ds = ds.assign_coords({gridlat+'_left':lat_left})
    coords = {
        'X':{'center':gridlon,'left':gridlon+'_left'},
        'Y':{'center':gridlat,'outer':gridlat+'_left'}
    }
    xgrid = Grid(ds, coords=coords, periodic=periodic)

    # Get horizontal distances
    dlonG = xgrid.diff(ds[gridlon], 'X', boundary_discontinuity=360)
    dlonC = xgrid.diff(ds[gridlon+'_left'], 'X', boundary_discontinuity=360)

    dlatG = xgrid.diff(ds[gridlat], 'Y', boundary='fill', fill_value=np.nan)
    dlatC = -xgrid.diff(ds[gridlat+'_left'], 'Y', boundary='fill', fill_value=np.nan)

    ds['dxG'], ds['dyG'] = _degrees_to_meters(dlonG, dlatG, ds[gridlon], ds[gridlat])
    ds['dxC'], ds['dyC'] = _degrees_to_meters(dlonC, dlatC, ds[gridlon], ds[gridlat])
    
    # Get vertical distances
    ds['dz'] = ds['lev_bounds'].diff('bnds').squeeze()
    
    # get area
    ds['areacello'] = ds['dxC']*ds['dyC']
    
    # Regenerate grid
    coords = {
        'X':{'center':gridlon,'outer':gridlon+'_left'},
        'Y':{'center':gridlat,'outer':gridlat+'_left'},
        'Z':{'center':gridlev,'outer':gridlev+'_outer'}
    }
    metrics = {
        'X':['dxC','dxG'],
        'Y':['dyC','dyG'],
        'Z':['dz'],
        ('X','Y'):['areacello']
    }
    
    xgrid = Grid(ds,coords=coords,metrics=metrics,periodic=periodic)
    
    return ds,xgrid

## I need to get `vo` for `CESM2-FV2.gn.historical.Omon.r1i1p1f1`

In [8]:
cat = col.search(
    source_id = 'CESM2-FV2',
    member_id = 'r1i1p1f1',
    experiment_id = 'historical',
    variable_id= ['vo'],
    grid_label = 'gn'
)
ddict = cat.to_dataset_dict(zarr_kwargs={'consolidated':True, 'use_cftime':True},preprocess=combined_preprocessing, aggregate=False)
ddict = merge_variables(ddict)

def pick_first_member(ds_list, **kwargs):
    return ds_list[0]

ddict = combine_datasets(
    ddict,
    pick_first_member,
    match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


In [9]:
ddict['CESM2-FV2.gn.historical.Omon']

<xarray.Dataset>
Dimensions:        (time: 1980, lev: 60, y: 384, x: 320, vertex: 4, bnds: 2)
Coordinates:
  * y              (y) int32 1 2 3 4 5 6 7 8 ... 377 378 379 380 381 382 383 384
  * x              (x) int32 1 2 3 4 5 6 7 8 ... 313 314 315 316 317 318 319 320
    lat            (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    lat_verticies  (y, x, vertex) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
  * lev            (lev) float64 5.0 15.0 25.0 ... 4.875e+03 5.125e+03 5.375e+03
    lev_bounds     (lev, bnds) float32 dask.array<chunksize=(60, 2), meta=np.ndarray>
    lon            (y, x) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    lon_verticies  (y, x, vertex) float32 dask.array<chunksize=(384, 320, 4), meta=np.ndarray>
  * time           (time) object 1850-01-15 13:00:00 ... 2014-12-15 12:00:00
    time_bounds    (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lon_bounds     (bnds, y, x) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    lat_bounds     (bnds, y, x) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
Dimensions without coordinates: vertex, bnds
Data variables:
    vo             (time, lev, y, x) float32 dask.array<chunksize=(3, 60, 384, 320), meta=np.ndarray>
Attributes: (12/49)
    Conventions:             CF-1.7 CMIP-6.2
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    674885.0
    branch_time_in_parent:   10950.0
    case_id:                 1559
    ...                      ...
    variant_info:            CMIP6 CESM2-FV2 historical experiment (1850-2014...
    variant_label:           r1i1p1f1
    netcdf_tracking_ids:     hdl:21.14100/bf4d5d07-2512-469f-a66d-81d81e469b8...
    version_id:              v20191120
    intake_esm_varname:      None
    intake_esm_dataset_key:  CMIP.NCAR.CESM2-FV2.historical.r1i1p1f1.Omon.vo....

# Regrid `ddict['CESM2-FV2.gn.historical.Omon']`

In [10]:
# Regrid
targetgrid_ds = xe.util.grid_global(1.0, 1.0)
targetgrid_ds['lon'] = targetgrid_ds['lon']+180
targetgrid_ds['lon_b'] = targetgrid_ds['lon_b']+180
newcoords = {'x':targetgrid_ds['lon'][0,:],'y':targetgrid_ds['lat'][:,0]}

dd_vo={}
for name,item in ddict.items():
    regridder = xe.Regridder(item, targetgrid_ds, 'bilinear', 
                         periodic=True, ignore_degenerate=True)
    ds_regridded = regridder(item).assign_coords(**newcoords).chunk({'time':120,'lev':1})
    dd_vo[name]=ds_regridded
list(dd_vo.keys())

['CESM2-FV2.gn.historical.Omon']

In [11]:
dd_vo['CESM2-FV2.gn.historical.Omon']

<xarray.Dataset>
Dimensions:      (time: 1980, lev: 60, y: 180, x: 360, bnds: 2)
Coordinates:
  * lev          (lev) float64 5.0 15.0 25.0 ... 4.875e+03 5.125e+03 5.375e+03
    lev_bounds   (lev, bnds) float32 dask.array<chunksize=(1, 2), meta=np.ndarray>
  * time         (time) object 1850-01-15 13:00:00 ... 2014-12-15 12:00:00
    time_bounds  (time, bnds) object dask.array<chunksize=(120, 2), meta=np.ndarray>
    lon          (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
    lat          (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
  * x            (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * y            (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
Dimensions without coordinates: bnds
Data variables:
    vo           (time, lev, y, x) float32 dask.array<chunksize=(120, 1, 180, 360), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

# Add `vo` to `dd_CESM2FV2`

In [14]:
ds_CESM2FV2 = dd_CESM2FV2['CESM2-FV2.gr.historical.Omon']
ds_CESM2FV2['vo'] = dd_vo['CESM2-FV2.gn.historical.Omon'].vo
dd_CESM2FV2

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1234: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]


{'CESM2-FV2.gr.historical.Omon': <xarray.Dataset>
 Dimensions:      (lev: 33, time: 239, y: 180, x: 360, bnds: 2)
 Coordinates:
   * lev          (lev) float64 0.0 10.0 20.0 30.0 ... 4.5e+03 5e+03 5.5e+03
   * time         (time) object 1850-01-15 13:00:00 ... 1869-11-15 00:00:00
     time_bounds  (time, bnds) object dask.array<chunksize=(239, 2), meta=np.ndarray>
     lev_bounds   (lev, bnds) float64 dask.array<chunksize=(33, 2), meta=np.ndarray>
     lon          (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
     lat          (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
   * x            (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
   * y            (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
 Dimensions without coordinates: bnds
 Data variables:
     hfds         (y, x) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
     fgco2        (y, x) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
     

# Apply functions to get more grid information

In [22]:
dd = {}
target = np.linspace(26, 28, 60)

for name, ds in dd_CESM2FV2.items():
    
    # Get center and outer z coords
    ds['lev'] = ds['lev_bounds'].mean('bnds').values
    ds['lev_outer'] = np.append(ds['lev_bounds'].isel(bnds=0),ds['lev_bounds'].isel(lev=-1,bnds=-1))

    # Generate xgcm grid
    # I think it is ok to overwrite xgrid since everything I need xgrid for is in this cell block
    ds,xgrid = get_xgcm_special(ds,periodic='X')
    
    # Create a 3d version of dz
    ds['dz3d']=ds['dz']*xr.ones_like(ds['vo'])

    # Interpolate thetao onto lev_outer
    ds['sigma0_outer3d'] = xgrid.interp(ds['sigma0'], 'Z', boundary='fill')
    ds['sigma0_outer2d'] = xgrid.interp(ds['sigma0'], 'Y', boundary='fill')
    
    # Need to chunk in lev_outer dimension otherwise transform gets upset
    ds = ds.chunk({'lev_outer':-1})
    dd[name] = ds


dd[name]

<xarray.Dataset>
Dimensions:         (lev: 33, time: 239, y: 180, x: 360, bnds: 2,
                     lev_outer: 34, x_left: 360, y_left: 181)
Coordinates:
  * lev             (lev) float64 2.5 10.0 20.0 32.5 ... 4.5e+03 5e+03 5.375e+03
  * time            (time) object 1850-01-15 13:00:00 ... 1869-11-15 00:00:00
    time_bounds     (time, bnds) object dask.array<chunksize=(239, 2), meta=np.ndarray>
    lev_bounds      (lev, bnds) float64 dask.array<chunksize=(33, 2), meta=np.ndarray>
    lon             (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
    lat             (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
  * x               (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * y               (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lev_outer       (lev_outer) float64 0.0 5.0 15.0 ... 5.25e+03 5.5e+03
  * x_left          (x_left) float64 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * y_left          (y_left) float64 -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
Dimensions without coordinates: bnds
Data variables: (12/21)
    hfds            (y, x) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
    fgco2           (y, x) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
    so              (lev, y, x) float32 dask.array<chunksize=(33, 180, 360), meta=np.ndarray>
    talk            (lev, y, x) float32 dask.array<chunksize=(33, 180, 360), meta=np.ndarray>
    intpp           (y, x) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
    thetao          (lev, y, x) float32 dask.array<chunksize=(33, 180, 360), meta=np.ndarray>
    ...              ...
    dyC             (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    dz              (lev) float64 dask.array<chunksize=(33,), meta=np.ndarray>
    areacello       (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    dz3d            (lev, time, y, x) float64 dask.array<chunksize=(7, 120, 180, 360), meta=np.ndarray>
    sigma0_outer3d  (lev_outer, y, x) float64 dask.array<chunksize=(34, 180, 360), meta=np.ndarray>
    sigma0_outer2d  (lev, y_left, x) float64 dask.array<chunksize=(33, 1, 360), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

<xarray.DataArray 'dissic' (lev: 33, y: 180, x: 360)>
dask.array<open_dataset-fd8b59d5381aba48edfb6bbc980e8f72dissic, shape=(33, 180, 360), dtype=float32, chunksize=(33, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 2.5 10.0 20.0 32.5 ... 4e+03 4.5e+03 5e+03 5.375e+03
    lon      (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
    lat      (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * y        (y) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5

# Calculate

## calculate `vdz` by multiplying `vo` by `dz`

In [30]:
ds = dd['CESM2-FV2.gr.historical.Omon']
# ds['vo'] = ds.vo.mean('time', keep_attrs = True)
dd[name].vo

<xarray.DataArray 'vo' (lev: 33, y: 180, x: 360)>
dask.array<mean_agg-aggregate, shape=(33, 180, 360), dtype=float32, chunksize=(26, 180, 360), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 2.5 10.0 20.0 32.5 ... 4e+03 4.5e+03 5e+03 5.375e+03
    lon      (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
    lat      (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
  * x        (x) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * y        (y) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5

In [32]:
ds['vdz'] = ds.vo * ds.dz
# # v_transport
dd[name]

<xarray.Dataset>
Dimensions:         (lev: 33, time: 239, y: 180, x: 360, bnds: 2,
                     lev_outer: 34, x_left: 360, y_left: 181)
Coordinates:
  * lev             (lev) float64 2.5 10.0 20.0 32.5 ... 4.5e+03 5e+03 5.375e+03
  * time            (time) object 1850-01-15 13:00:00 ... 1869-11-15 00:00:00
    time_bounds     (time, bnds) object dask.array<chunksize=(239, 2), meta=np.ndarray>
    lev_bounds      (lev, bnds) float64 dask.array<chunksize=(33, 2), meta=np.ndarray>
    lon             (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
    lat             (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
  * x               (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * y               (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lev_outer       (lev_outer) float64 0.0 5.0 15.0 ... 5.25e+03 5.5e+03
  * x_left          (x_left) float64 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * y_left          (y_left) float64 -90.0 -89.0 -88.0 -87.0 ... 88.0 89.0 90.0
Dimensions without coordinates: bnds
Data variables: (12/22)
    hfds            (y, x) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
    fgco2           (y, x) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
    so              (lev, y, x) float32 dask.array<chunksize=(33, 180, 360), meta=np.ndarray>
    talk            (lev, y, x) float32 dask.array<chunksize=(33, 180, 360), meta=np.ndarray>
    intpp           (y, x) float32 dask.array<chunksize=(180, 360), meta=np.ndarray>
    thetao          (lev, y, x) float32 dask.array<chunksize=(33, 180, 360), meta=np.ndarray>
    ...              ...
    dz              (lev) float64 dask.array<chunksize=(33,), meta=np.ndarray>
    areacello       (x, y) float64 dask.array<chunksize=(360, 180), meta=np.ndarray>
    dz3d            (lev, time, y, x) float64 dask.array<chunksize=(7, 120, 180, 360), meta=np.ndarray>
    sigma0_outer3d  (lev_outer, y, x) float64 dask.array<chunksize=(34, 180, 360), meta=np.ndarray>
    sigma0_outer2d  (lev, y_left, x) float64 dask.array<chunksize=(33, 1, 360), meta=np.ndarray>
    vdz             (lev, y, x) float64 dask.array<chunksize=(7, 180, 360), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

## Transform `vdz` to `sigma0`

In [44]:
# Interpolate thetao onto lev_outer
ds['sigma0_outer'] = xgrid.interp(ds['sigma0'], 'Z', boundary='extend')
# Need to chunk in lev_outer dimension otherwise transform gets upset
ds = ds.chunk({'lev_outer':-1})

In [57]:
dummy['CESM2-FV2.gr.historical.Omon'] = {}
da = xr.Dataset()
da['vdz'] = ds.vdz
da['sigma0_outer'] = ds.sigma0_outer
dummy['CESM2-FV2.gr.historical.Omon'] = da
dummy

{'CESM2-FV2.gr.historical.Omon': <xarray.Dataset>
 Dimensions:       (lev: 33, y: 180, x: 360, lev_outer: 34)
 Coordinates:
   * lev           (lev) float64 2.5 10.0 20.0 32.5 ... 4.5e+03 5e+03 5.375e+03
     lon           (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
     lat           (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
   * x             (x) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
   * y             (y) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
   * lev_outer     (lev_outer) float64 0.0 5.0 15.0 ... 4.75e+03 5.25e+03 5.5e+03
 Data variables:
     vdz           (lev, y, x) float64 dask.array<chunksize=(7, 180, 360), meta=np.ndarray>
     sigma0_outer  (lev_outer, y, x) float64 dask.array<chunksize=(34, 180, 360), meta=np.ndarray>}

In [58]:
target = np.linspace(26, 28, 30)
xgrid_sigma = Grid(da, coords = {'Z': {'center':'lev','outer':'lev_outer'}}, periodic=False)

vdz_sigma = xgrid_sigma.transform(da['vdz'],
                             'Z',
                             target,
                             method='linear',
                             target_data=ds['sigma0_outer'])
vdz_sigma

ValueError: operand to apply_ufunc has required core dimensions ['lev'], but some of these dimensions are absent on an input variable: ['lev']

In [60]:
da.sigma0_outer

<xarray.DataArray 'sigma0_outer' (lev_outer: 34, y: 180, x: 360)>
dask.array<rechunk-merge, shape=(34, 180, 360), dtype=float64, chunksize=(34, 180, 360), chunktype=numpy.ndarray>
Coordinates:
    lon        (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
    lat        (y, x) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>
  * x          (x) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * y          (y) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lev_outer  (lev_outer) float64 0.0 5.0 15.0 ... 4.75e+03 5.25e+03 5.5e+03